# pytwoway example

In [1]:
# Add pytwoway to system path, do not run this
# import sys
# sys.path.append('../../..')

In [2]:
# Import the pytwoway package 
# (Make sure you have installed it using pip install pytwoway)
import pytwoway as tw

## Simulate some data

The package contains functions to simulate data. We use this here to keep things simple. If you have your own data, you can import it. Load it as a pandas dataframe and use it as an input. 

As you can see, we will need the following required columns in our data: 

 - `wid`: the worker identifier
 - `fid`: the firm identifier
 - `year`: the time
 - `comp`: the outcome variable, in our case compensation

In [3]:
# For the example, we simulate data
sim_data = tw.SimTwoWay().sim_network()
display(sim_data)

,i,t,k,alpha,psi,spell,freq,j,move,y
0,0,1,1,-0.430727,-0.908458,1,2,17,False,-0.004062
1,0,2,1,-0.430727,-0.908458,1,2,17,False,-0.293843
2,0,3,0,-0.430727,-1.335178,2,2,7,True,-2.475346
3,0,4,0,-0.430727,-1.335178,2,2,7,False,-3.363936
4,0,5,2,-0.430727,-0.604585,3,1,36,True,-2.608726
...,...,...,...,...,...,...,...,...,...,...
49995,9999,1,3,-0.430727,-0.348756,1,2,58,False,-0.353585
49996,9999,2,3,-0.430727,-0.348756,1,2,58,False,-1.366538
49997,9999,3,1,-0.430727,-0.908458,2,1,20,True,-2.436596
49998,9999,4,1,-0.430727,-0.908458,3,1,19,True,-1.700477


## Create a TwoWay object using your data

In [4]:
# We need to specify a column dictionary to make sure columns are named correctly
# You can also manually update column names yourself
col_name_dict = {
    'i': 'i',    # Specify the column name for the worker identifier 
    'j': 'j',    # Specify the column name for the firm identifier 
    'y': 'y',  # Specify the column name for the compensation variable
    't': 't'   # Specify the column name for the time variable
}

# Create the TwoWay object that will do all the heavy lifting
tw_net = tw.TwoWay(data=sim_data, formatting='long', col_dict=col_name_dict)

## Now we can run the FE estimator

In [5]:
## Optional Parameters ##
fe_params = {
    'ncore': 1, # Number of cores to use
    'batch': 1, # Batch size to send in parallel
    'ndraw_pii': 50, # Number of draw to use in approximation for leverages
    'ndraw_tr': 5, # Number of draws to use in approximation for traces
    'check': False, # Whether to compute the non-approximated estimates as well
    'hetero': False, # Whether to compute the heteroskedastic estimates
    'out': 'res_fe.json', # Filepath for fe results
    'con': False, # Computes the smallest eigen values, this is the filepath where these results are saved
    'logfile': '', # Log output to a logfile
    'levfile': '', # File to load precomputed leverages
    'statsonly': False # Save data statistics only
}

# Estimate the fixed effect decomposition
tw_net.fit_fe(user_fe=fe_params)

100%|██████████| 5/5 [00:00<00:00, 242.93it/s]


## We can also run the CRE estimator

In [6]:
## Optional Parameters ##
cre_params = {
    'ncore': 1, # Number of cores to use
    'ndraw_tr': 5, # Number of draws to use in approximation for traces
    'ndp': 50, # Number of draw to use in approximation for leverages
    'out': 'res_cre.json', # Filepath for cre results
    'posterior': False, # Whether to compute the posterior variance
    'wobtw': False # Sets between variation to 0, pure RE when computing cre
}
KMeans_params = { # These parameters are specifically for the KMeans algorithm
                  # Read more at https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
    'n_clusters': 10,
    'init': 'k-means++',
    'n_init': 500,
    'max_iter': 300,
    'tol': 0.0001,
    'precompute_distances': 'deprecated',
    'verbose': 0,
    'random_state': None,
    'copy_x': True,
    'n_jobs': 'deprecated',
    'algorithm': 'auto'
}
cluster_params = {
    'cdf_resolution': 10, # How many values to use to approximate the cdf when clustering
    'grouping': 'quantile_all',
        # How to group the cdfs when clustering
        # ('quantile_all' to get quantiles from entire set of data,
        #     then have firm-level values between 0 and 1;
        # 'quantile_firm_small' to get quantiles at the firm-level
        #     and have values be compensations if small data;
        # 'quantile_firm_large' to get quantiles at the firm-level
        #     and have values be compensations if large data,
        #     note that this is up to 50 times slower than 'quantile_firm_small'
        #     and should only be used if the dataset is too large to copy
        #     into a dictionary)
    'year': None, # If None, uses entire dataset when clustering;
                  # If int, gives year of data to consider when clustering
    'user_KMeans': KMeans_params
}

# Estimate the cre model
tw_net.fit_cre(user_cre=cre_params, user_cluster=cluster_params)

## Finally, we can investigate the results

In [7]:
display(tw_net.summary_fe())
display(tw_net.summary_cre())

{'var_fe': 0.5997301987660023,
 'cov_fe': 0.17580097176675977,
 'var_ho': 0.5938941220301615,
 'cov_ho': 0.1783081225830144,
 'var_y': 1.9801317055315406}

{'var_y': 1.9801317055315406,
 'var_bw': 0.5833941085601263,
 'cov_bw': 0.18293340911401856,
 'var_tot': 0.5812933186878522,
 'cov_tot': 0.18263757032833403}